In [1]:
import pulp as p
import numpy as np
import pandas as pd

In [7]:
#FTL Capacity Utilization Problem for KL PAL & SBY

types = ['BATH-ZWIT-001F','DIAP-HAPP-001O','DIAP-HAPP-001P','DIAP-MAMY-017A-JSMA','GIFT-ZWIT-003A','SUSU-SGM0-008A-NEW']
typesvol = np.array([0.0194775,0.04144,0.041796,0.047212,0.036736,0.03565])
typesvalue = np.array([87,800,800,1796,7,282])

supply = ['BJI', 'KL','SIP']
                      
#supplycap          #BJI   #KL  #SIP
supplyqty = np.array([[20,100,70], #sku1
                      [35,47,200],
                      [78,65,422],
                      [250,27,300],
                      [100,500,10],
                      [500,275,500]])#sku n

supplyexist = supplyqty > 0

supplyvalue = supplyexist.astype(int)*typesvalue.reshape(-1, 1)


demand = ['CDD', 'CDDLong', 'Wingbox']

                      #dem1 #dem2 #dem3
demandvol = np.array([14,  25,  50])#total volume of truck

demandcost = np.array([5500000, 8500000, 12000000])


supplyconstraint = p.makeDict([types, supply], supplyqty)

demandconstraint = dict(zip(demand,demandvol))

demandcostdict = p.makeDict([demand], demandcost)

supplyvaluedict = p.makeDict([types, supply], supplyvalue)

#typesconstraint = p.makeDict([types], typesvol)

typesconstraint = dict(zip(types, typesvol))

prob = p.LpProblem('Trucks_Selection', p.LpMaximize)

sup_qty_vars = p.LpVariable.dicts("items", (types, supply), lowBound = 0, cat = 'Integer')

no_of_trucks_vars = p.LpVariable.dicts("vehicles", (demand), lowBound = 0, cat = 'Integer')


#optimization function: maximise item to send by looking at their value (target location's forecast - available stock)
# while selecting the cheapest transportation method, hence the formula = summing all value then less transport cost
prob += p.lpSum([sup_qty_vars[t][s]*supplyvaluedict[t][s] for t in types for s in supply])\
            - p.lpSum([no_of_trucks_vars[d]*demandcostdict[d]/1000 for d in demand])


for t in types:
    for s in supply:
        prob += p.lpSum([sup_qty_vars[t][s]]) <= supplyconstraint[t][s] 

#truck capacity must be more than total volume combined, max capacity utilization of trucks of 97%
prob += p.lpSum([no_of_trucks_vars[d]*demandconstraint[d]*0.97 for d in demand]) >= p.lpSum([sup_qty_vars[t][s]*typesconstraint[t] for t in types for s in supply])

#truck must be at least 90% full
prob += p.lpSum([no_of_trucks_vars[d]*demandconstraint[d]*0.9 for d in demand]) <= p.lpSum([sup_qty_vars[t][s]*typesconstraint[t] for t in types for s in supply])

#at least 1 truck must be selected
prob += p.lpSum([no_of_trucks_vars[d] for d in demand]) >= 0.1

prob.solve()

print('maximum utility: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])


qtydict = {}
vehicledict = {}

for item in prob.variables():
    print(item.name,' = ',item.varValue)
    if 'items' in item.name: 
        qtydict[item.name] = item.varValue
    if 'vehicle' in item.name:
        vehicledict[item.name] = item.varValue


skuvoldict = pd.Series(data = np.array(list(qtydict.values())) * np.repeat(typesvol,3), index = qtydict.keys()) 

vehicledict = pd.Series(data=np.array(list(vehicledict.values()))*demandvol, index=demand)

print('items qty requested: ', supplyqty.sum())

print('items qty selected: ', sum(qtydict.values()))

print('items utilization: ', sum(qtydict.values())/supplyqty.sum())

print('volume of items to ship: ',sum(skuvoldict))

print('total capacity of vehicles:', sum(vehicledict))

print('volume utilization: ', sum(skuvoldict)/sum(vehicledict))


maximum utility:  2059292.0
Solution Status: Optimal
items_BATH_ZWIT_001F_BJI  =  20.0
items_BATH_ZWIT_001F_KL  =  100.0
items_BATH_ZWIT_001F_SIP  =  70.0
items_DIAP_HAPP_001O_BJI  =  35.0
items_DIAP_HAPP_001O_KL  =  47.0
items_DIAP_HAPP_001O_SIP  =  200.0
items_DIAP_HAPP_001P_BJI  =  78.0
items_DIAP_HAPP_001P_KL  =  65.0
items_DIAP_HAPP_001P_SIP  =  422.0
items_DIAP_MAMY_017A_JSMA_BJI  =  250.0
items_DIAP_MAMY_017A_JSMA_KL  =  27.0
items_DIAP_MAMY_017A_JSMA_SIP  =  300.0
items_GIFT_ZWIT_003A_BJI  =  0.0
items_GIFT_ZWIT_003A_KL  =  260.0
items_GIFT_ZWIT_003A_SIP  =  0.0
items_SUSU_SGM0_008A_NEW_BJI  =  500.0
items_SUSU_SGM0_008A_NEW_KL  =  275.0
items_SUSU_SGM0_008A_NEW_SIP  =  500.0
vehicles_CDD  =  0.0
vehicles_CDDLong  =  1.0
vehicles_Wingbox  =  2.0
items qty requested:  3499
items qty selected:  3149.0
items utilization:  0.8999714204058302
volume of items to ship:  121.247979
total capacity of vehicles: 125.0
volume utilization:  0.969983832


In [ ]:
#Using Actual Data

klsbymilp = pd.read_csv('kl_sby_input.csv')

#FTL Capacity Utilization Problem for KL SBY

maxcapacity = 0.95
mincapacity = 0.8

types = klsbymilp.sku.values.astype(str)
typesvol = klsbymilp['volume(m3)'].to_numpy()
typesvalue = klsbymilp['util_value'].to_numpy()

supply = ['BJI', 'KL','SIP']
                      
#supplycap      
supplyqty = klsbymilp[['BJI(carton)','KLJKT(carton)','SIP(carton)']].to_numpy()

supplyexist = supplyqty > 0

supplyvalue = supplyexist.astype(int)*typesvalue.reshape(-1, 1)


demand = ['CDD', 'CDDLong', 'Wingbox']

                      #dem1 #dem2 #dem3
demandvol = np.array([16,  22,  38])#total volume of truck

demandcost = np.array([4500000, 5500000, 8000000])


supplyconstraint = p.makeDict([types, supply], supplyqty)

demandconstraint = dict(zip(demand,demandvol))

demandcostdict = p.makeDict([demand], demandcost)

supplyvaluedict = p.makeDict([types, supply], supplyvalue)

#typesconstraint = p.makeDict([types], typesvol)

typesconstraint = dict(zip(types, typesvol))

prob = p.LpProblem('Trucks_Selection', p.LpMaximize)

sup_qty_vars = p.LpVariable.dicts("items", (types, supply), lowBound = 0, cat = 'Integer')

no_of_trucks_vars = p.LpVariable.dicts("vehicles", (demand), lowBound = 0, cat = 'Integer')


#optimization function: maximise item to send by looking at their value (target location's forecast - available stock)
# while selecting the cheapest transportation method, hence the formula = summing all value then less transport cost
prob += p.lpSum([sup_qty_vars[t][s]*supplyvaluedict[t][s] for t in types for s in supply])\
            - p.lpSum([no_of_trucks_vars[d]*demandcostdict[d]*100 for d in demand])


for t in types:
    for s in supply:
        prob += p.lpSum([sup_qty_vars[t][s]]) <= supplyconstraint[t][s] 

#truck capacity must be more than total volume combined, max capacity utilization of trucks of 97%
prob += p.lpSum([no_of_trucks_vars[d]*demandconstraint[d]*maxcapacity for d in demand]) >= p.lpSum([sup_qty_vars[t][s]*typesconstraint[t] for t in types for s in supply])

#truck must be at least 90% full
prob += p.lpSum([no_of_trucks_vars[d]*demandconstraint[d]*mincapacity for d in demand]) <= p.lpSum([sup_qty_vars[t][s]*typesconstraint[t] for t in types for s in supply])

#at least 1 truck must be selected
prob += p.lpSum([no_of_trucks_vars[d] for d in demand]) >= 1

prob.solve()

print('maximum utility: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

if p.LpStatus[prob.status] == 'Infeasible':
    raise SystemExit("Not Enough Quantity to be Shipped")


qtydict = {}
vehicledict = {}

vehicleused = {}

for item in prob.variables():
    print(item.name,' = ',item.varValue)
    if 'items' in item.name: 
        qtydict[item.name] = item.varValue
    if 'vehicle' in item.name:
        vehicledict[item.name] = item.varValue
        vehicleused[item.name] = item.varValue


skuvoldict = pd.Series(data = np.array(list(qtydict.values())) * np.repeat(typesvol,3), index = qtydict.keys()) 

vehicledict = pd.Series(data=np.array(list(vehicledict.values()))*demandvol, index=demand)

vehicleused = pd.Series(data=np.array(list(vehicleused.values())), index=demand)

print('items qty requested: ', supplyqty.sum())

print('items qty selected: ', sum(qtydict.values()))

print('items utilization: ', sum(qtydict.values())/supplyqty.sum())

print('volume of items to ship: ',sum(skuvoldict))

print('total capacity of vehicles:', sum(vehicledict))

print('volume utilization: ', sum(skuvoldict)/sum(vehicledict))

klsby_opt = klsbymilp

optim_qty = np.array(list(qtydict.values())).reshape(len(types),len(supply))

klsby_opt['BJI(carton) Optimized'] = optim_qty[:,0]
klsby_opt['KLJKT(carton) Optimized'] = optim_qty[:,1]
klsby_opt['SIP(carton) Optimized'] = optim_qty[:,2]

klsby_opt.to_excel('kl_sby_output.xlsx', index=False)